# Word2vec preprocessing

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make SkipGramBatcher class which returns two numpy tensors with word indices. It should be possible to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size,)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. The result of your work should represent that your batch has a proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
         ['used', 'against', 'working', 'class'],
         ['against', 'early', 'class', 'radicals'],
         ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']

# Skip-Gram

indices_to_words(x_batch) = ['against', 'early', 'working', 'class']

indices_to_words(labels_batch) = ['used', 'working', 'early', 'radicals']]

```

If you struggle with something, ask your neighbor. If it is not obvious for you, probably someone else is looking for the answer too. And in contrast, if you see that you can help someone - do it! Good luck!

In [1]:
from collections import Counter
import numpy as np


class SkipGramBatcher():
    def __init__(self, corpus, window_size=2, batch_size=3, vocab_size=5000):
        self.window_size = window_size
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.corpus = np.asarray(corpus, dtype=str)

        # 1. Count all word occurencies.
        counted_words = Counter(corpus).most_common(self.vocab_size)
        # create dict using dict comprehension
        self.idx_to_word = {idx: word for idx, (word, count) in enumerate(counted_words)}
        self.word_to_idx = {word: idx for idx, (word, count) in enumerate(counted_words)}

        # append '<UNK>' token to dictionaries
        last_idx = len(self.idx_to_word)
        self.idx_to_word[last_idx] = '<UNK>'
        self.word_to_idx['<UNK>'] = self.idx_to_word

    def words_to_indexes(self, words):
        unk_index = self.word_to_idx['<UNK>']
        idxes = [self.word_to_idx.get(word, unk_index) for word in words]
        return idxes

    def indexes_to_words(self, indexes):
        words = [self.idx_to_word[index] for index in indexes]
        return words

    # iteration should return batch and labels
    # https://stackoverflow.com/questions/19151/build-a-basic-python-iterator

    # __iter__ should reset inner counter
    def __iter__(self):
        self.batch_start_pos = 0
        return self

    def get_random_sample(self, center_id):
        left_window = np.arange(max(0, center_id - self.window_size),
                                center_id)
        right_window = np.arange(center_id + 1,
                                 min(center_id + self.window_size + 1,len(self.corpus)))
        window = np.concatenate((left_window, right_window))
        position = np.random.choice(window)
        return self.words_to_indexes([self.corpus[position]])

    def __next__(self):
        if self.batch_start_pos >= len(self.corpus):
            raise StopIteration
        else:
            batch_position_in_corpus = np.arange(self.batch_start_pos,
                                                 min(self.batch_start_pos + self.batch_size,
                                                     len(self.corpus)))
            x_batch = np.asarray(self.words_to_indexes(self.corpus[batch_position_in_corpus]))
            # draw a word from window of a selected word
            y_batch = np.asarray([self.get_random_sample(selected_word_position)
                                  for selected_word_position in batch_position_in_corpus]).flatten()
            # for selected_word_position in batch_position_in_corpus:
            #     y_batch.append(self.get_random_sample(selected_word_position))
            self.batch_start_pos += self.batch_size
            return x_batch, y_batch


In [2]:
# text = []
# with open('./data/text8', 'r') as text8:
#     text = text8.read().split()

In [3]:
batcher = SkipGramBatcher(text, window_size=2)
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'class', 'other']

In [4]:
for idx, (x_batch, y_batch) in enumerate(batcher):
    if (idx % 10000) == 0:
        print(idx)

NameError: name 'a' is not defined